# Getting started with Agent 

Agent is an AI agent framework that utilises Large Language Models to incorporate structured reasoning in decision making processes. Agent is designed such that it may be adapted for use in an arbitrary task defined by the user. Here, a task is simply an environment in which the agent is attempting to acheive a particular goal. Agent comes with several inbuilt tasks that are used to demonstrate the agent capabilities. These tasks are:
* Alfworld
* BabyAI
* Data Science
* GSM8K
* HotpotQA
* HumanEval
* Webshop


As mentioned, Agent uses structured reasoning to make decisions within an environment. The method of reasoning used by the agent is highly customisable, and must be specified by the user at runtime. Details on how reasoning methods may be specified can be found below.

## Reasoning methods

Agent operates by using an LLM to observe and generate actions for a given task. At the start of each timestep, the LLM agent takes an initial input consisting of information about the task, current state of the enviroment, and information from the agent memory. At the end of the step, the agent outputs an external action to the environment. The process of generating external actions from the initial input on a given timestep happens according to the reasoning method defined to the agent. Reasoning methods provide the framework for the agent to generate a chain of thought about the best action to take, based on the observation of the environment.

For a given reasoning method, the LLM within the agent is prompted several times in a sequential manner. A sequence of "reasoning steps" constitutes an environment step. Each reasoning step is aimed to encourage the agent to generate thoughts about its input before outputting an action to the external environment.

Agent comes packaged with several core inbuilt reasoning methods. The core methods included are:
* `direct`: prompt the agent to directly output an action to environment
* `zs-cot`: prompt the agent to generate thoughts before giving an action
* `fs-cot`: prompt the agent to generate thoughts before giving an action, with chain of thought example provided in the prompt
* `fs-cot-react`: prompt the agent to only generate a thought first, then action on subsequent reasoning step
* `fs-cot-reflect`: prompt the agent to reflect on past trajectory before giving action
* `fs-cot-sc`: runs fs-cot several times and selects most consistent action
* `fs-cot-zerostep-reflect`: prompts the agent to give an action, and then reflect of whether that action is the best choice
* `fs-least2most`: prompt the agent to decompose problem into sub-problems before generating an action

Each method is associated with a .yaml configuration file in /agent/configs/method folder. This file is intended to provide arguments to the Agent API in order to structure the reasoning flow on each step during the task. Below, we look at an example of a simple method: `fs-cot-zerostep-reflect`. 



### fs-cot-zerostep-reflect method example

The .yaml configuration file for fs-cot-zerostep-reflect is defined as:

```
agent:
  main_flow:
    _target_: agent.commands.SequentialFlow
    sequence:
      - _target_: agent.commands.ZeroStepReflect
  prompt_builder:
    default_kwargs:
      cot_type: few_shot_cot
      reflect_cot_type: few_shot_cot
```

There are three main things defined here: the type of reasoning flow, the commands (functions) defined within the flow, and the chain of thought (cot) type. Here we define that the agent uses a `SequentialFlow` decision making process on each timestep. In this case, we have one command in the overarching sequence: the `ZeroStepReflect` command. The `ZeroStepReflect` command is itself actually a composite function defined as a sequential flow consisting of two sub commands : `ConsiderAction`, `ReflectOnPlannedAction` and `ExecutedPlannedAction`. This means that on each timestep the agent uses a sequential reasoning process as follows:

1. Agent takes in an initial input (in the form of a natural language prompt), and outputs a proposed action.
2. The LLM agent is prompted again to reflect on whether the action it has chosen is the correct choice based on the task observation and memory, and either confirm its current choice or select a new one.
3. The new action choice is taken as the final action, and output to the environment.

Note that we define the cot_type as few_shot_cot. This means that, each time the LLM is prompted, it is asked to give an output that takes the form of a chain of thoughts, culminating in a final response.


## Prompt templates

As discussed above, on each step of a reasoning flow, the LLM agent is prompted to output a response. These prompts are structured according to predefined prompt templates, defined within jinja files found in src/agent/prompts/templates. Each command in the flow uses a given predefined template, and fills in the template with relevant information such as the current observation of the environment, or relevant details from the agent's memory. As an example, we look at how the prompt is structured when the `ConsiderAction` command is called. This function uses the external_action.jinja template, which is give as:

```
{% include "system_prompt.jinja" %} 

[[ USER ]]
{%- if cot_type in ["few_shot", "few_shot_cot", "react"] %}
{% include "context_example.jinja" %}
{%- endif %}

{%- set subproblem = memory.retrieve({"subproblem": 1.0}) %}
{%- if subproblem %}
{% include "subproblem_example.jinja" %}
{% endif %}


{% include "trajectory.jinja" %}

{%- set thought = memory.retrieve({"thought": 1.0}) %}
{%- if thought %}

Here are some thoughts you had before:

{{thought}}
{%- endif %}

{%- if subproblem %}

Here are some subproblems you had before:

{{subproblem}}

{%- endif %}

{%- if memory.retrieve({"reflect": 1.0}) is not none %}
Here are your past reflections and plans:
{{memory.retrieve({"reflect": 1.0})}}
{%- endif %}


{%- if memory.retrieve({"available_actions": 1.0}) %}

{% include "available_actions.jinja" %}
{%- endif %}

{% if cot_type in ["zero_shot_cot", "few_shot_cot"] %}
{% include "cot_prompt.jinja" %}
{%- elif cot_type in ["few_shot_least_to_most"] %}
{%- if subproblem %}
{% include "subproblem.jinja" %}
{%- endif %}
{%- else %}
{% include "direct_prompt.jinja" %}
{%- endif %}
```

Breaking down the file above, this template includes:
* A system_prompt, comprising information about about the task
* A context_example, providing examples of solved tasks
* Sub-task details if they exist
* The current environment trajectory, comprising the observation history of the environment.
* Past thoughts about the environment



## Running an example task

We now run a task example using a specific method and LLM. We use the gsm8k task as an example, using the fs-cot-zerostep-reflect method. Run the cell below in order to run the task.

In [ ]:
!python ../src/agent/start.py task=gsm8k method=fs-cot-zerostep-reflect llm@agent.llm=deepseek_coder